In [ ]:
# %pip install requests
# %pip install pandas
# %pip install urllib

# %pip install python-dotenv
# %pip install python-decouple

# %pip install psycopg2-binary
# %pip install sqlalchemy

# %pip install pyspark[all]

# libary included to solve not module pyspark found
%pip install findspark

In [1]:
# imports
import os
import requests
import urllib.parse

# DB connector and libraries
import psycopg2

import findspark
findspark.init()
# Pyspark utils
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col

In [ ]:
print(os.getcwd())

## Iniciar la sesión de spark

In [14]:
class Spark_Session:
    
    _driver_path: str = ""
    
    def __init__(self):
        
#         Define environment variables
        self._driver_path = os.getcwd() + "/driver_jdbc/postgresql-42.2.27.jre7.jar"

        os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {self._driver_path} --jars {self._driver_path} pyspark-shell'
        os.environ['SPARK_CLASSPATH'] = self._driver_path
        # Start spark session
        self._start_spark_session()
        
    def _start_spark_session(self):
        self._spark: SparkSession = SparkSession.builder \
            .master("local") \
            .appName("Conexion entre Pyspark y Redshift") \
            .config("spark.jars", self._driver_path) \
            .config("spark.executor.extraClassPath", self._driver_path) \
            .getOrCreate()
    
    def return_session(self):
        """
        Función que retorna la sessión de spark creada al instanciar la clase.
        """
        return self._spark
            
spark = Spark_Session()
spark_session = spark.return_session()


In [7]:
def url_constructor(url: str, **kargs) -> str:

    return f"{url}?{urllib.parse.urlencode(kargs)}"

## Obtenemos la información desde la url

In [16]:
# Vamos a consultar la información historica diaria que nos emite Binance sin limites inicio y fin

data = requests.get( \
                url_constructor("https://api.binance.com/api/v3/klines", symbol="BTCUSDT", interval="1d")
        ).json()

headers = ("Open High", 
           "Low", 
           "Close", 
           "Volume", 
           "Close time", 
           "Quote asset volume", 
           "Number of trades",
           "Taker buy base asset volume",
           "Taker buy quote asset volume", 
           "Ignore")

df = spark_session.createDataFrame(data, [x for x in headers])

In [18]:
df.printSchema()


root
 |-- Open High: long (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Close time: string (nullable = true)
 |-- Quote asset volume: string (nullable = true)
 |-- Number of trades: long (nullable = true)
 |-- Taker buy base asset volume: string (nullable = true)
 |-- Taker buy quote asset volume: long (nullable = true)
 |-- Ignore: string (nullable = true)
 |-- _11: string (nullable = true)
 |-- _12: string (nullable = true)

